In [60]:
import sys
from sqlalchemy import create_engine, text

sys.path.append("C:/Users/esniede/Documents/github/YGwater/dev")
import pyquacache as pq
db_url = pq.config.db_url_prod

engine = create_engine(db_url)
with engine.connect() as connection:
        table_result = connection.execute(
            text("""SELECT rr.reference_id, rr.valid_from
                        FROM raster_series_index rsi
                        JOIN rasters_reference rr ON rsi.raster_series_id = rr.raster_series_id
                        JOIN rasters r ON r.reference_id = rr.reference_id
                        WHERE rsi.model = 'ERA5_land'
                        ORDER BY rr.valid_from, r.reference_id"""))

import pandas as pd
pd.DataFrame(table_result)




ResourceClosedError: This Connection is closed

In [77]:
import numpy as np
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt

# Try using ST_AsGDALRaster with GTiff format instead
subset_query = """
   SELECT 
      r.reference_id,
      ST_AsGDALRaster(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326)), 'GTiff') as clipped_raster,
      ST_AsText(ST_Envelope(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326)))) as clipped_footprint
   FROM spatial.rasters r
   WHERE r.reference_id = (
      SELECT reference_id 
      FROM spatial.rasters_reference 
      LIMIT 1
   )
   AND ST_Intersects(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326))
"""

try:
    with engine.connect() as connection:
        subset_result = connection.execute(text(subset_query))
        subset_df = pd.DataFrame(subset_result.fetchall(), columns=subset_result.keys())
        print(f"Query returned {len(subset_df)} rows")
        
        # If we have raster data, read it with rasterio
        if not subset_df.empty and 'clipped_raster' in subset_df.columns:
            for idx, row in subset_df.iterrows():
                # Get the binary raster data
                raster_data = row['clipped_raster']
                
                # Use MemoryFile to read the raster data
                with MemoryFile(raster_data) as memfile:
                    with memfile.open() as dataset:
                        # Read raster properties
                        print(f"Raster {row['reference_id']}:")
                        print(f"  Shape: {dataset.shape}")
                        print(f"  CRS: {dataset.crs}")
                        print(f"  Bounds: {dataset.bounds}")
                        
                        # Read the actual raster data
                        raster_array = dataset.read(1)  # Read first band
                        print(f"  Data type: {raster_array.dtype}")
                        print(f"  Min/Max values: {np.nanmin(raster_array):.2f} / {np.nanmax(raster_array):.2f}")
                        
                        # Visualize the raster
                        plt.figure(figsize=(10, 8))
                        plt.imshow(raster_array, cmap='viridis', interpolation='nearest')
                        plt.colorbar(label='Values')
                        plt.title(f'Raster {row["reference_id"]}')
                        plt.show()

except Exception as e:
    print(f"Error with ST_AsGDALRaster: {e}")
    
    # Fallback: try getting raster values as array
    fallback_query = """
       SELECT 
          r.reference_id,
          ST_DumpValues(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326)), 1) as raster_values,
          ST_Width(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326))) as width,
          ST_Height(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326))) as height
       FROM spatial.rasters r
       WHERE r.reference_id = (
          SELECT reference_id 
          FROM spatial.rasters_reference 
          LIMIT 1
       )
       AND ST_Intersects(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326))
    """
    
    with engine.connect() as connection:
        fallback_result = connection.execute(text(fallback_query))
        fallback_df = pd.DataFrame(fallback_result.fetchall(), columns=fallback_result.keys())
        print("Fallback query results:")
        print(fallback_df)


Error with ST_AsGDALRaster: (psycopg2.errors.InternalError_) rt_raster_to_gdal: Could not load the output GDAL driver

[SQL: 
   SELECT 
      r.reference_id,
      ST_AsGDALRaster(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326)), 'GTiff') as clipped_raster,
      ST_AsText(ST_Envelope(ST_Clip(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326)))) as clipped_footprint
   FROM spatial.rasters r
   WHERE r.reference_id = (
      SELECT reference_id 
      FROM spatial.rasters_reference 
      LIMIT 1
   )
   AND ST_Intersects(r.rast, ST_MakeEnvelope(-141, 60, -124, 69, 4326))
]
(Background on this error at: https://sqlalche.me/e/20/2j85)
Fallback query results:
   reference_id                                      raster_values  width  \
0         11339  [[None, None, None, None, None, None, None, No...    171   

   height  
0      91  


In [80]:
fallback_df.raster_values[:]

0    [[None, None, None, None, None, None, None, No...
Name: raster_values, dtype: object

In [64]:

from pathlib import Path
import geopandas as gpd

data_dir = Path("H:/esniede/data")

basins = gpd.read_file(data_dir / "WSC/MDA_ADP_09/MDA_ADP_09_DrainageBasin_BassinDeDrainage.geojson")

In [19]:
import pandas as pd

pd.DataFrame(table_result)

,raster_series_id,model,start_datetime,end_datetime,last_new_raster,source_fx,source_fx_args,parameter,last_issue,raster_type_id,...,param_description,created_by,modified_by,created,modified,parameter_id,aggregation_type_id,media_id,z_value,z_units
0,9,HRDPA,2023-11-22 18:00:00+00:00,2026-02-06 12:00:00+00:00,2026-02-06 22:10:58.742886+00:00,downloadHRDPA,"[{'clip': 'YT', 'parameter': 'APCP-Accum6h_Sfc'}]",accumulated precipitation (all types) at surface,NaT,2,...,"Accumulated precip at ground, all types",postgres,postgres,2025-08-12 18:05:42.421677+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
1,10,HRDPS,2025-10-17 13:00:00+00:00,2026-02-08 07:00:00+00:00,2026-02-06 21:43:33.207241+00:00,downloadHRDPS,"[{'clip': 'YT', 'parameter': 'APCP-Accum1h_Sfc'}]",accumulated precip (all types) 1 hour,2026-02-06 18:00:00+00:00,1,...,"Accumulated precip at ground, all types",postgres,postgres,2025-08-12 18:05:42.421677+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
2,16,CaLDAS,2025-09-30 21:00:00+00:00,2026-02-06 00:00:00+00:00,2026-02-06 04:15:39.681893+00:00,downloadCaLDAS,"{'hrs': '0', 'clip': 'YT', 'parameter': 'SnowW...",snow water equivalent,NaT,2,...,Snow water equivalent (column of water),admin,postgres,2025-10-21 23:32:14.894414+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
3,13,ERA5_land,1950-01-01 00:00:00+00:00,2026-02-01 00:00:00+00:00,2026-02-06 00:01:36.798061+00:00,downloadERA5,"[{'hrs': '0', 'key': '5815cfa9-2642-46bd-9a7f-...",snow water equivalent,NaT,2,...,Snow water equivalent (column of water),postgres,postgres,2025-08-12 18:05:42.421677+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
4,12,HRDPS,2025-10-16 19:00:00+00:00,2026-02-08 07:00:00+00:00,2026-02-06 21:46:25.125386+00:00,downloadHRDPS,"[{'clip': 'YT', 'parameter': 'WEARN-Accum1h_Sf...",accumulated rain 1 hour,2026-02-06 18:00:00+00:00,1,...,Accumulated rain at ground,postgres,postgres,2025-08-12 18:05:42.421677+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
5,11,HRDPS,2025-10-21 13:00:00+00:00,2026-02-08 19:00:00+00:00,2026-02-06 21:50:28.595003+00:00,downloadHRDPS,"[{'clip': 'YT', 'parameter': 'WEASN-Accum1h_Sf...",accumulated snow 1 hour,2026-02-06 18:00:00+00:00,1,...,Accumulated snow at ground,postgres,postgres,2025-08-12 18:05:42.421677+00:00,2026-02-06 22:11:02.304121+00:00,None,None,None,None,None
